<a href="https://colab.research.google.com/github/ritunagpal/AndroidSwipeLayout/blob/master/5_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install langchain-core langchain-community faiss-cpu tiktoken PyPDF opendatasets sentence-transformers --quiet
%pip install python-dotenv langchain-together --quiet

In [30]:
import dotenv
import os
from langchain.document_loaders import TextLoader

os.environ["TOGETHER_API_KEY"] = "4ed3130d9b14026bbded56d76f0586a9b832a4366373b2c41de78538adede286"
dotenv.load_dotenv()

False

In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import warnings
from langchain_together import TogetherEmbeddings
from langchain_community.llms import Together

# warnings.filterwarnings("ignore")

In [7]:
pdf_loader = PyPDFLoader('./1706.03762.pdf')
pages = pdf_loader.load_and_split()

In [32]:
text_loader = TextLoader('./David Cameron Speech 1.txt',encoding="utf8")
text_pages = text_loader.load_and_split()

In [33]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1024,
    chunk_overlap  = 128,
    length_function = len,
)
chunks = text_splitter.split_documents(text_pages)

In [34]:
chunks[14]

Document(metadata={'source': './David Cameron Speech 1.txt'}, page_content='Would going it alone really give us more control over our affairs, or would we soon find that actually we had less, and that we had given up a secure future for one beset by years of uncertainty and trouble with no way back?\nWould going it alone open up new opportunities, or would it in fact close them down and narrow our options?”\nStronger in the World')

In [35]:
Embeddings = TogetherEmbeddings(
    model="togethercomputer/m2-bert-80M-8k-retrieval")

vectorStore = FAISS.from_texts([str(chunk) for chunk in chunks], Embeddings)

In [37]:
## Without LLM
question = """
What has been created in the UK over the last six years?
"""
docs = vectorStore.similarity_search(question, k = 2)

docs

[Document(page_content="page_content='The Treasury has calculated that the cost to every household in Britain would be as high as £4,300 by 2030 if we leave. £4,300.\nThe overwhelming weight of independent opinion – from the International Monetary Fund to the OECD, from the London School of Economics to the Institute for Fiscal Studies – also supports the fact that Britain will suffer an immediate economic shock, and then be permanently poorer for the long-term.\nThe evidence is clear: we will be better off in, and poorer if we leave.\nAs Charles Dunstone, the founder of Carphone Warehouse, an entrepreneur not averse to risk, has said: ‘In my experience there are calculated risks, there are clever risks, and there are unnecessary and dangerous risks. And from all I can conclude, Brexit sits firmly in the latter camp.’' metadata={'source': './David Cameron Speech 1.txt'}"),
 Document(page_content="page_content='We are the product of our long history – of the decision of our forebears, o

In [38]:
prompt = """
    Using this information:
    \n
    Context: {context}
    \n
    Answer the following:
    \n
    Question: {question}
    \n
    Answer:\n
    """
prompt = ChatPromptTemplate.from_template(prompt)

In [39]:
retriever = vectorStore.as_retriever(kwargs={"k": 3})

llm = Together(model="meta-llama/Meta-Llama-3-70B-Instruct-Turbo")


In [40]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       retriever=retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={
                                           'prompt': prompt,
                                           'verbose': True
                                           },
                                       verbose=True
                                       )

In [41]:
question = "What has been created in the UK over the last six years? "

result = qa_chain({"query": question})



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
    Using this information:
    

    Context: page_content='The Treasury has calculated that the cost to every household in Britain would be as high as £4,300 by 2030 if we leave. £4,300.
The overwhelming weight of independent opinion – from the International Monetary Fund to the OECD, from the London School of Economics to the Institute for Fiscal Studies – also supports the fact that Britain will suffer an immediate economic shock, and then be permanently poorer for the long-term.
The evidence is clear: we will be better off in, and poorer if we leave.
As Charles Dunstone, the founder of Carphone Warehouse, an entrepreneur not averse to risk, has said: ‘In my experience there are calculated risks, there are clever risks, and there are unnecessary and dangerous risks. And from all I can conclude, Brexit sits firmly in the latter camp.

In [42]:
result["query"]

'What has been created in the UK over the last six years? '

In [43]:
print(result["result"])

 Almost 2.4 million jobs have been created in the UK over the last six years.


In [44]:
print(result["source_documents"][0].page_content)

page_content='The Treasury has calculated that the cost to every household in Britain would be as high as £4,300 by 2030 if we leave. £4,300.
The overwhelming weight of independent opinion – from the International Monetary Fund to the OECD, from the London School of Economics to the Institute for Fiscal Studies – also supports the fact that Britain will suffer an immediate economic shock, and then be permanently poorer for the long-term.
The evidence is clear: we will be better off in, and poorer if we leave.
As Charles Dunstone, the founder of Carphone Warehouse, an entrepreneur not averse to risk, has said: ‘In my experience there are calculated risks, there are clever risks, and there are unnecessary and dangerous risks. And from all I can conclude, Brexit sits firmly in the latter camp.’' metadata={'source': './David Cameron Speech 1.txt'}


In [45]:
print("Your Question: ", result["query"])
print()
print("Your answer: ", result["result"])
print()
print("Sources: ", result["source_documents"])

Your Question:  What has been created in the UK over the last six years? 

Your answer:   Almost 2.4 million jobs have been created in the UK over the last six years.

Sources:  [Document(page_content="page_content='The Treasury has calculated that the cost to every household in Britain would be as high as £4,300 by 2030 if we leave. £4,300.\nThe overwhelming weight of independent opinion – from the International Monetary Fund to the OECD, from the London School of Economics to the Institute for Fiscal Studies – also supports the fact that Britain will suffer an immediate economic shock, and then be permanently poorer for the long-term.\nThe evidence is clear: we will be better off in, and poorer if we leave.\nAs Charles Dunstone, the founder of Carphone Warehouse, an entrepreneur not averse to risk, has said: ‘In my experience there are calculated risks, there are clever risks, and there are unnecessary and dangerous risks. And from all I can conclude, Brexit sits firmly in the latter

In [46]:
docs[0].page_content

"page_content='The Treasury has calculated that the cost to every household in Britain would be as high as £4,300 by 2030 if we leave. £4,300.\nThe overwhelming weight of independent opinion – from the International Monetary Fund to the OECD, from the London School of Economics to the Institute for Fiscal Studies – also supports the fact that Britain will suffer an immediate economic shock, and then be permanently poorer for the long-term.\nThe evidence is clear: we will be better off in, and poorer if we leave.\nAs Charles Dunstone, the founder of Carphone Warehouse, an entrepreneur not averse to risk, has said: ‘In my experience there are calculated risks, there are clever risks, and there are unnecessary and dangerous risks. And from all I can conclude, Brexit sits firmly in the latter camp.’' metadata={'source': './David Cameron Speech 1.txt'}"